In [1]:
# Correct latent for F2100W (June-28 exposures)

In [2]:
import sep
from astropy.io import fits
from glob import glob
from astropy.wcs import WCS
from astropy.table import Table
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.convolution import Tophat2DKernel
from scipy.ndimage import binary_dilation

import os
os.environ['CRDS_PATH']= '/Users/users/gyang/gyang/crds_cache/jwst_pub'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds-pub.stsci.edu'

from jwst.pipeline import Image3Pipeline
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3 import Asn_Lv3Image

%run -i '../../tools/astrometry/srcor.ipynb'
%run -i 'super_bg.ipynb'

In [3]:
# Get the June-28 file names
fnames = sorted( glob('../data/reduced/jw01345052001_02101_0000?_mirimage_cal_stprm_bgsub.fits') )
# List to save detected source catalogs and segmentation 
cats, segs, whts, wcss = [], [], [], []
# Iterate over each file
for idx, fname in enumerate(fnames):
    # Perform source detection
    src, dat, wht, bkg, seg = det_src(fname, sigma_threshold=1., minarea=10, cr_sigclip=4,
                                      clean_edge=False)
    # Convert x,y to ra, dec
    ras, decs = [], []
    wcs = WCS(fits.getheader(fname, 'sci'))
    for cor in wcs.pixel_to_world(src['x']-1, src['y']-1):
        ras.append( cor.ra.deg )
        decs.append( cor.dec.deg )
    cat = Table( [src['id'], src['x'], src['y'], ras, decs],
                  names=['id', 'x', 'y', 'ra', 'dec'] )
    print(len(cat))
    # Save the results
    segs.append(seg)
    cats.append(cat)
    whts.append(wht)
    wcss.append(wcs)
    # Write out segmentation and wheight maps
    fits.writeto('../data/persistence/seg_all_%d.fits' %(idx+1), seg,
                 header=fits.getheader(fname, 'sci'), overwrite=True)    
    fits.writeto('../data/persistence/wht_%d.fits' %(idx+1), wht,
                 header=fits.getheader(fname, 'sci'), overwrite=True)    

2022-09-12 16:13:56,035 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'datfix' made the change 'Set DATE-BEG to '2022-06-28T07:49:01.462' from MJD-BEG.
Set DATE-AVG to '2022-06-28T07:57:33.457' from MJD-AVG.
Set DATE-END to '2022-06-28T08:06:05.452' from MJD-END'.
  warnings.warn(

2022-09-12 16:13:56,037 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.412891 from OBSGEO-[XYZ].
Set OBSGEO-B to   -37.794096 from OBSGEO-[XYZ].
Set OBSGEO-H to 1730760512.408 from OBSGEO-[XYZ]'.
  warnings.warn(



296


2022-09-12 16:13:57,378 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'datfix' made the change 'Set DATE-BEG to '2022-06-28T08:07:28.725' from MJD-BEG.
Set DATE-AVG to '2022-06-28T08:16:00.720' from MJD-AVG.
Set DATE-END to '2022-06-28T08:24:32.715' from MJD-END'.
  warnings.warn(

2022-09-12 16:13:57,379 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.410718 from OBSGEO-[XYZ].
Set OBSGEO-B to   -37.795705 from OBSGEO-[XYZ].
Set OBSGEO-H to 1730772551.630 from OBSGEO-[XYZ]'.
  warnings.warn(



132


2022-09-12 16:13:58,551 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'datfix' made the change 'Set DATE-BEG to '2022-06-28T08:25:58.804' from MJD-BEG.
Set DATE-AVG to '2022-06-28T08:34:30.799' from MJD-AVG.
Set DATE-END to '2022-06-28T08:43:02.794' from MJD-END'.
  warnings.warn(

2022-09-12 16:13:58,552 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.10/site-packages/astropy/wcs/wcs.py:725: FITSFixedWarning: 'obsfix' made the change 'Set OBSGEO-L to   -73.408661 from OBSGEO-[XYZ].
Set OBSGEO-B to   -37.797228 from OBSGEO-[XYZ].
Set OBSGEO-H to 1730783969.615 from OBSGEO-[XYZ]'.
  warnings.warn(



128


In [4]:
# If a source is matched to any sources in the other two catalogs
# it's considered as a real source 
# We only have to do for the 1st dither, as its latent should be the strongest
idx0 = 0
# Get the index of the other two catalogs
oth_idxs = list(range(3))
oth_idxs.remove(idx0)
idx1, idx2 = oth_idxs[0], oth_idxs[1]
# Match cat0 with cat1 and cat2, respectively 
rad = 0.5
cat0_cat1_idxs, cat1_cat0_idxs, cat01_ds = match(cats[idx0]['ra'], cats[idx0]['dec'],
                                          cats[idx1]['ra'], cats[idx1]['dec'], rad, opt=0)
cat0_cat2_idxs, cat2_cat0_idxs, cat02_ds = match(cats[idx0]['ra'], cats[idx0]['dec'],
                                          cats[idx2]['ra'], cats[idx2]['dec'], rad, opt=0)
# Get real sources (1-based ID)
real_ids = np.unique( np.append(cats[idx0]['id'][cat0_cat1_idxs],
                                cats[idx0]['id'][cat0_cat2_idxs]) )
# Futher assume a source as real if it low weight in the other two dithers
wht_thresh = 0.2
for row in cats[idx0]:
    # If it's already a real source
    if row['id'] in real_ids: continue
    # Get the detector coordinates in the other two dithers
    x1, y1 = wcss[idx1].world_to_pixel( SkyCoord( [row['ra']], [row['dec']], unit='deg' ) )
    x1, y1 = round(x1[0]), round(y1[0])
    x2, y2 = wcss[idx2].world_to_pixel( SkyCoord( [row['ra']], [row['dec']], unit='deg' ) )
    x2, y2 = round(x2[0]), round(y2[0])    
    # Check if the position has low weights in both wht maps
    if (y1>=0) and (x1>=0) and (y1<whts[idx1].shape[0]) and (x1<whts[idx1].shape[1]):
        wht1 = whts[idx1][y1, x1]
    else: wht1 = np.nan
    if (y2>=0) and (x2>=0) and (y2<whts[idx2].shape[0]) and (x2<whts[idx2].shape[1]):
        wht2 = whts[idx2][y2, x2]
    else: wht2 = np.nan    
    wht1_thresh = wht_thresh*np.nanmax(whts[idx1])
    wht2_thresh = wht_thresh*np.nanmax(whts[idx2])
    
    if (np.isnan(wht1) or wht1<wht1_thresh) and (np.isnan(wht2) or wht2<wht2_thresh):
        real_ids = np.append(real_ids, row['id'])
        
# Remove sources near the brightest persistence 
real_Sidxs = []
for src_Sidx, real_id in enumerate(real_ids):
    # Get the source position
    src_idx = np.where( cats[idx0]['id'] == real_id )[0][0]
    x, y = cats[idx0]['x'][src_idx], cats[idx0]['y'][src_idx]
    # real sources should be far away from the brightest persistence 
    if (x-481)**2 + (y-567)**2 > 107**2: real_Sidxs.append(src_Sidx)
real_ids = real_ids[real_Sidxs]
        
# Clean the real sources in segmentation map
seg = segs[idx0].copy()
for real_id in real_ids:
    seg[seg==real_id] = 0 

# Write out the segmentation map
fits.writeto('../data/persistence/seg_fake_%d.fits' %(idx0+1), seg,
             header=fits.getheader(fnames[idx0], 'sci'), overwrite=True)

# Do segmentation dilation as a master mask map
#footprint = Tophat2DKernel(radius=1)
#seg = binary_dilation(seg, footprint.array).astype(int)
fits.writeto('../data/persistence/latent_mask.fits', seg, overwrite=True)

Multi-one match:  45  sources
Multi-one match:  39  sources


In [7]:
import jwst
from jwst import datamodels
%run -i 'run_tkreg.ipynb'

# Create an association 
asn = dict( asn_from_list.asn_from_list(fnames, rule=Asn_Lv3Image, 
            product_name='l3_results', asn_type="image3") )
# Initialize stage 3 pipeline
# In the following, we use ".copy()" as input to avoid the input being changed 
img3 = Image3Pipeline()
# Load the data models
dms = datamodels.open(asn)

tkreg_res = run_tkreg(dms, img3, cr_sigclip=1, run2=False)

# match bkg
img3.skymatch.subtract = True
img3.skymatch.skymethod = 'local'
res = img3.skymatch.run(tkreg_res.copy())

# Reject cosmic-ray 
res = img3.outlier_detection.run(res)

# Apply the mask
#mask_map = fits.getdata('../data/persistence/latent_mask.fits')
#mask = mask_map!=0
#for exp_idx in range(len(res)):
#    res[exp_idx].data[mask] = 0
#    res[exp_idx].dq[mask] = 11 # the flag is rather some arbitrary bad 
#    res[exp_idx].err[mask] = np.inf
#    res[exp_idx].var_rnoise[mask] = np.inf

res = img3.resample(res)
# Output 
res.save('../data/persistence/merged_latent_not_masked.fits')

# Clean up cache
os.system('rm *.fits')

2022-09-12 16:20:55,315 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2022-09-12 16:20:55,317 - stpipe - CRITICAL - config: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'assign_mtwcs', 'search_output_file': True, 'input_dir': None, 'class': ''}


2022-09-12 16:20:55,318 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2022-09-12 16:20:55,321 - stpipe - CRITICAL - config: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': None, 'save_catalogs': False, 'catalog_format': 'ecsv', 'kernel_fwhm': 2.5, 'snr_threshold': 10.0, 'brightest': 200, 'peakmax': None, 'enforce_user_order': False, 'expand_re

2022-09-12 16:21:01,231 - stpipe.Image3Pipeline.tweakreg - INFO - XSH: -0.00343105  YSH: -0.0204054    ROT: 0.0453898    SCALE: 1
2022-09-12 16:21:01,231 - stpipe.Image3Pipeline.tweakreg - INFO - 
2022-09-12 16:21:01,232 - stpipe.Image3Pipeline.tweakreg - INFO - FIT RMSE: 0.130104   FIT MAE: 0.10454
2022-09-12 16:21:01,232 - stpipe.Image3Pipeline.tweakreg - INFO - Final solution based on 21 objects.
2022-09-12 16:21:01,273 - stpipe.Image3Pipeline.tweakreg - INFO - Aligning image catalog 'GROUP ID: jw01345052001_02101_00003_mirimage_cal_stprm_bgsub' to the reference catalog.
2022-09-12 16:21:01,392 - stpipe.Image3Pipeline.tweakreg - INFO - Matching sources from 'jw01345052001_02101_00003_mirimage_cal_stprm_bgsub' catalog with sources from the reference 'jw01345052001_02101_00001_mirimage_cal_stprm_bgsub' catalog.
2022-09-12 16:21:01,393 - stpipe.Image3Pipeline.tweakreg - INFO - Computing initial guess for X and Y shifts...
2022-09-12 16:21:01,396 - stpipe.Image3Pipeline.tweakreg - INFO 

2022-09-12 16:21:03,643 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter fillval: INDEF
2022-09-12 16:21:03,643 - stpipe.Image3Pipeline.outlier_detection - INFO - Driz parameter weight_type: ivm
2022-09-12 16:21:03,783 - stpipe.Image3Pipeline.outlier_detection - INFO - 1 exposures to drizzle together
2022-09-12 16:21:04,483 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (1024, 1032) --> (1042, 1057)
2022-09-12 16:21:05,169 - stpipe.Image3Pipeline.outlier_detection - INFO - Exposure jw01345052001_02101_00001_mirimage_cal_stprm_outlier_i2d.fits saved to file
2022-09-12 16:21:05,176 - stpipe.Image3Pipeline.outlier_detection - INFO - 1 exposures to drizzle together
2022-09-12 16:21:06,339 - stpipe.Image3Pipeline.outlier_detection - INFO - Drizzling (1024, 1032) --> (1042, 1057)
2022-09-12 16:21:07,234 - stpipe.Image3Pipeline.outlier_detection - INFO - Exposure jw01345052001_02101_00002_mirimage_cal_stprm_outlier_i2d.fits saved to file
2022-09-12 16:21:07,241 

0

0